In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

from crewai import Agent,LLM,Crew,Task
from crewai_tools import TavilySearchTool
from crewai.tools import tool



In [3]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.2,
    api_key=os.getenv("GROQ_API_KEY")
    )


In [4]:
search_tool = TavilySearchTool(max_results=3)

In [ ]:
# research a new project idea, do the research on market demand, risk, and potential return on investment.

# Define the Manager AI
manager_agent = Agent(
    role="Project Research Manager",
    goal="Oversee the project research and ensure timely, high-quality responses.",
    backstory="""An experienced project manager responsible
                 for ensuring project research.""",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# Define the Technical Support AI
market_demand_agent = Agent(
    role="Market Demand Analyst",
    goal="Write market demand content.",
    backstory="""A skilled market demand analyst who
                 writes market demand content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
risk_analysis_agent = Agent(
    role="Risk Analysis Analyst",
    goal="Write risk analysis content.",
    backstory="""A risk analysis analyst who
                 writes fiction content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
return_on_investment_agent = Agent(
    role="Return on Investment Analyst",
    goal="Write return on investment content.",
    backstory="""A return on investment analyst who
                writes return on investment content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)


In [ ]:
manager_task = Task(
    description="""Oversee the project research on {project_title} and ensure timely, high-quality responses.""",
    expected_output="A manager-approved response ready to be sent as an article on {project_title}.",
    agent=manager_agent, 
)

market_demand_task = Task(
    description="""Analyze the market demand for the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=market_demand_agent, 
)

risk_analysis_task = Task(
    description="""Analyze the risk of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=risk_analysis_agent, 
)

return_on_investment_task = Task(
    description="""Analyze the return on investment of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=return_on_investment_agent, 
)

final_report_task = Task(
    description="""Review the final responses from the 
                   market demand, risk analysis, and return on investment agents
                   and create a final report.""",
    expected_output="""A comprehensive report on the project title '{project_title}'
    containing the market demand, risk analysis, and return on investment.""",
    agent=manager_agent,
)

In [ ]:
from crewai import Crew, Process

project_research_crew = Crew(
    agents=[market_demand_agent, risk_analysis_agent, return_on_investment_agent],
    
    tasks=[market_demand_task, risk_analysis_task, return_on_investment_task, final_report_task],
    
    manager_agent=manager_agent,
    
    process=Process.hierarchical,
    
    verbose=True,
)

In [ ]:
inputs = {"project_title": "Amazon INC AMZN"}

result = project_research_crew.kickoff(inputs=inputs)

In [ ]:
# 2nd Manual Manager Agent

In [ ]:
# 1. Market Researcher Agent
market_researcher = Agent(
    role="Market Research Specialist",
    goal="Research target audience, competitors, and trending topics in the industry",
    backstory="""You are an expert market researcher with 10 years of experience.
    You know how to find valuable insights about audiences, trends, and competitors.
    You provide detailed, data-driven research reports.""",
    tools=[search_tool],
    llm=llm,
    verbose=True,
    max_tokens = 1000,
)

# 2. SEO Specialist Agent
seo_specialist = Agent(
    role="SEO Expert",
    goal="Identify high-value keywords and SEO optimization strategies",
    backstory="""You are a seasoned SEO professional who understands search algorithms.
    You find the perfect keywords that balance search volume with ranking difficulty.
    You provide actionable SEO recommendations.""",
    llm=llm,
    verbose=True,
    max_tokens = 1000,
)

# 3. Content Writer Agent
content_writer = Agent(
    role="Senior Content Writer",
    goal="Create engaging, well-structured blog posts that resonate with the audience",
    backstory="""You are a talented writer with expertise in creating compelling content.
    You know how to weave research and keywords into natural, engaging narratives.
    Your content drives engagement and conversions.""",
    llm=llm,
    verbose=True,
    max_tokens = 1000,
)

# 4. Quality Editor Agent
quality_editor = Agent(
    role="Editorial Quality Checker",
    goal="Review content for accuracy, completeness, and quality standards",
    backstory="""You are a meticulous editor with a keen eye for detail.
    You ensure every piece meets high standards for accuracy, clarity, and completeness.
    You're not afraid to send work back if it needs improvement.""",
    llm=llm,
    verbose=True,
    max_tokens = 1000,
)

# =========================
# TASKS DEFINITION
# =========================

# Task 1: Market Research
research_task = Task(
    description="""Research the topic: {topic}
    
    Provide:
    - Target audience demographics and pain points
    - Top 3 competitor articles on this topic
    - Current trends related to this topic
    - Key questions the audience is asking
    
    Make sure your research is comprehensive and recent.
    if require use search tool to perform the action """,
    agent=market_researcher,
    expected_output="Detailed market research report with audience insights, competitor analysis, and trends"
)

# Task 2: SEO Keyword Research
seo_task = Task(
    description="""Based on the market research, identify SEO opportunities for: {topic}
    
    Provide:
    - 5-10 primary and secondary keywords
    - Search intent for each keyword
    - Keyword difficulty and search volume estimates
    - Recommended content structure for SEO
    
    Ensure keywords are relevant and achievable.""",
    agent=seo_specialist,
    expected_output="SEO keyword strategy with search volumes, difficulty scores, and content recommendations"
)

# Task 3: Content Writing
writing_task = Task(
    description="""Write a comprehensive blog post about: {topic}
    
    Requirements:
    - Use the market research insights to address audience pain points
    - Naturally incorporate the SEO keywords identified
    - Create an engaging introduction and strong conclusion
    - Include practical examples and actionable tips
    - Aim for 1500-2000 words
    - Use clear headings and subheadings
    
    The content must be original, valuable, and well-structured.""",
    agent=content_writer,
    expected_output="Complete blog post draft with proper structure, SEO optimization, and engaging content"
)

# Task 4: Quality Check
quality_task = Task(
    description="""Review the blog post for quality and completeness.
    
    Check:
    - Does it address the audience pain points from research?
    - Are SEO keywords naturally incorporated?
    - Is the content comprehensive (500-1000 words)?
    - Is it well-structured with clear headings?
    - Are there practical examples and actionable tips?
    - Is the tone engaging and professional?
    
    If ANY requirement is not met, flag it clearly and request improvements.
    If everything looks good, approve the content.""",
    agent=quality_editor,
    expected_output="Quality assessment report with approval or specific improvement requests"
)

# =========================
# CREW WITH HIERARCHICAL PROCESS
# =========================

# # Option 1: Let CrewAI automatically create a manager
# content_crew = Crew(
#     agents=[market_researcher, seo_specialist, content_writer, quality_editor],
#     tasks=[research_task, seo_task, writing_task, quality_task],
#     process=Process.hierarchical,  # This enables hierarchical mode
#     verbose=True,
#     memory=True  # Enables crew memory for better context
# )

# Option 2: Explicitly define your own manager (uncomment to use)

manager_agent = Agent(
    role="Content Marketing Manager",
    goal="Coordinate the content creation process and ensure high-quality output",
    backstory='''You are an experienced content marketing manager who knows how to 
    orchestrate teams effectively. You delegate tasks wisely, validate outputs, 
    and ensure the final content meets all requirements. You're not afraid to 
    request revisions when needed.''',
    llm=llm,
    allow_delegation=True
)

content_crew = Crew(
    agents=[market_researcher, seo_specialist, content_writer, quality_editor],
    tasks=[research_task, seo_task, writing_task, quality_task],
    process=Process.hierarchical,
    manager_agent=manager_agent,  # Use your custom manager
    verbose=True,
    memory=True
)

# =========================
# EXECUTION
# =========================

if __name__ == "__main__":
    # Input topic
    inputs = {
        "topic": "How AI is Transforming Small Business Marketing in 2024"
    }
    
    print("\n🚀 Starting Content Marketing Crew (Hierarchical Process)...\n")
    print("=" * 70)
    
    # The manager will:
    # 1. Assign research to Market Researcher
    # 2. Validate research output
    # 3. Assign SEO task to SEO Specialist
    # 4. Validate SEO output
    # 5. Assign writing to Content Writer
    # 6. Assign quality check to Quality Editor
    # 7. If quality check fails -> Go back to research/writing as needed
    # 8. If quality check passes -> Complete the workflow
    
    result = content_crew.kickoff(inputs=inputs)
    
    print("\n" + "=" * 70)
    print("\n✅ FINAL OUTPUT:\n")
    print(result)